This notebook was created by Donna Faith Go.

In [18]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os

# data gathering
import yfinance as yf
import time
import pandas_datareader.data as web
from datetime import datetime, timedelta

# ignore warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## Closing Prices

## Stock Tickers

In [4]:
filepath = 'SSE companies list.pkl'
with open(filepath, 'rb') as f:
    companies_df = pickle.load(f)
companies_df.head()

,Company Name,Company Code
0,"SHANGHAI PUDONG DEVELOPMENT BANK CO., LTD.",600000
1,"Guangzhou Baiyun International Airport Co.,ltd.",600004
2,"Dongfeng Automobile Co.,LTD",600006
3,China World Trade Center Company Ltd.,600007
4,Beijing Capital Eco-Environment Protection Gro...,600008


In [5]:
tickers_list = companies_df['Company Code'].to_list()
tickers_list = [ticker + ".SS" for ticker in tickers_list]
print(tickers_list[:5])

['600000.SS', '600004.SS', '600006.SS', '600007.SS', '600008.SS']


## yfinance

In [19]:
# pull individual stock data from yfinance
def download_info_per_stock(ticker, verbose=False, 
                            start_date='2000-01-01', 
                            end_date='2026-01-01'):
        try:
            # get data for the ticker
            ticker_data = yf.download(
                ticker,
                start=start_date,
                end=end_date,
                progress=False
            )
            return pd.DataFrame(ticker_data)
            
        except Exception as e:
            if verbose:
                print(f"Error downloading batch {batch}: {e}")
            return None

# saving individual stock data
def save_info_per_stock(ticker_list, delay=1, 
                        verbose=False, override=False,
                        start_date='2000-01-01', 
                        end_date='2026-01-01'):
    
    # create the data folder
    os.makedirs("data", exist_ok=True)

    for i in range(0, len(ticker_list)):
        if verbose:
            print(f"Downloading for ticker: {ticker_list[i]}")

        # get the data for each stock
        ticker_data = download_info_per_stock(ticker_list[i])

        # saving data as a pkl file
        if ticker_data is not None and not ticker_data.empty:
            filepath = f"data/{ticker_list[i]}.pkl"

            if override or not os.path.exists(filepath):
                ticker_data.to_pickle(filepath)
                if verbose == True:
                    print(f"Saved data for {ticker_list[i]}.")
            else:
                if verbose == True:
                    print(f"Skipped {ticker_list[i]}.")
                

        # avoid rate limiting
        time.sleep(delay)

    print("Done downloading all data!")

In [ ]:
save_info_per_stock(tickers_list)


1 Failed download:
['600004.SS']: Timeout('Failed to perform, curl: (28) Resolving timed out after 10005 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['600037.SS']: Timeout('Failed to perform, curl: (28) Connection timed out after 10005 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['600120.SS']: SSLError('Failed to perform, curl: (35) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['600128.SS']: Timeout('Failed to perform, curl: (28) Operation timed out after 10003 milliseconds with 99547 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['600129.SS']: Timeout('Failed to perform, curl: (28) Resolving timed out after 10004 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1

## Disregarded Code

This code was disregarded because the files it generated were too big to be committed on the GitHub.

In [2]:
# declare start and end dates
start_date = '2000-01-01'
end_date = '2026-01-01'

def download_stocks_in_batches(tickers, batch_size=5, delay=1, verbose=False):
    """
    Download stock data in batches to avoid rate limiting
    """
    all_data = {}
    
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        if verbose:
            print(f"Downloading batch {i//batch_size + 1}: {batch}")
        
        try:
            # Download the batch
            batch_data = yf.download(
                batch,
                start=start_date,
                end=end_date,
                progress=False
            )
            
            # Extract closing prices for this batch
            if not batch_data.empty and 'Close' in batch_data.columns:
                closes = batch_data['Close']
                if isinstance(closes, pd.Series):
                    all_data[batch[0]] = closes
                else:
                    for ticker in closes.columns:
                        all_data[ticker] = closes[ticker]
                if verbose:
                    print(f"Successfully downloaded {len(batch)} stocks")
            else:
                print(f"No data returned for batch: {batch}")
            
        except Exception as e:
            print(f"Error downloading batch {batch}: {e}")
        
        # Add delay to avoid rate limiting
        if i + batch_size < len(tickers):
            if verbose:
                print(f"Waiting {delay} seconds before next batch...")
            time.sleep(delay)
    
    if all_data:
        return pd.DataFrame(all_data)
    else:
        return pd.DataFrame()

In [1]:
# # Download the closing prices
# closing_df = download_stocks_in_batches(
#     tickers_list, 
#     batch_size=5, 
#     delay=5
# )

# # removing unnecessary columns and rows
# closing_df.dropna(how='all', axis=1, inplace=True)
# closing_df.dropna(how='all', axis=0, inplace=True)

# # # save to pkl
# # if not closing_df.empty:
# #     closing_df.to_pickle('SSE companies closing prices.pkl')

# # cut into parts
# closing_df1 = closing_df.iloc[:, :int(np.floor(len(closing_df) / 2))]
# closing_df2 = closing_df.iloc[:, int(np.floor(len(closing_df) / 2)):]

# # save as pkl files
# if not closing_df1.empty and closing_df2.empty:
#     closing_df1.to_pickle('data/01 SSE companies closing prices.pkl')
#     closing_df2.to_pickle('data/02 SSE companies closing prices.pkl')

Note: The code above takes around 30-45 minutes to completely run. 